# <center> Teamfight Tactics, <br> and  Predicting the Chance <br> of Winning a Game </center>

#### <center>Samuel Kahan</center>

## <center>Introduction</center>

Teamfight Tactics (TFT) is a multiplayer auto-battler game developed by Riot Games. An 'auto-battler' is a type of strategy game where players build a team of characters and then let the game play out battles automatically between different teams. TFT combines aspects of other games like chess, poker, and League of Legends into one. The game is broken up into rounds. There are three types of rounds:

    - PVP Rounds
    - PVE Rounds
    - Carousels

During PVP and PVE rounds, each player is presented with a shop of five units which they can purchase. They can then place these units onto their battlefield. Buying three of the same unit combines them into one silver unit, and three silver units combine to make a golden unit, each stronger than the last. Units have many stats, but the most important ones are generally HP and mana. When a unit's HP hits 0, it's eliminated from the battlefiend. When a unit's mana hits 100%, it casts it's ability. Each unit has an ability which has some sort of effect, whether that be dealing damage to an enemy unit or healing your own units. Mana is gained from dealing damage and taking damage, but can also be generated by items (more on items soon). 

The difference between PVP and PVE rounds is the enemy. The 'P' in PVP stands for player, and the 'E' in PVE stands for enemy. While this may seem strange as the other players ARE your enemies, PVE is a term in video games that generally refers to playing against NPC's, or non-player characters. In PVP rounds, players go up against other players, and watch their teams fight. The loser loses some of their total HP, which starts at 100. Once you hit 0 total HP, you are eliminated. In PVE rounds, players go up against NPC's. Defeating these NPC's drops gold, items, and other loot. Items can be put on units to increase their stats, and can be combined into stronger items that have additional effects. Choosing which units to put your items on and which items to build are integral parts of the strategy of TFT.

In carousels, each player picks a unit and item from a group of unit/item combinations. The units walk around in a circle (where the word carousel comes from), and two players can pick a unit at a time. A unit is picked by simply walking into it, and players with lower total health pick first.

During each game, there are three points where the player is offered one of three 'augments' to pick from. These augments have varying effects, from buffing your unit's strength, to giving gold and items. Along with these augments, every unit has different 'traits' that can also buff your team. For example, Dark Star units gain offenstive stats whenever an ally unit is eliminated, but you must play three different Dark Star units to activate this effect. These trait effects generally get stronger the more units you play. 

Of course there is much more to the game than just these things, but this is the general idea; make the strongest board you can make at any given point, and try to minimize the total HP you lose throughout the game. In this notebook, we will explore data from TFT's seventh set, analyzing different team combinations. We will use this information to attempt to predict player placements (out of 8) when using different unit/item combinations. 

In [22]:
# import pandas as pd
# from pandas.io.json import json_normalize
# import requests
# import json

# api_key = "RGAPI-14ceaf26-f9c2-45a2-bb8e-0c33d1e75610"
# summoner_name = "Yu Yevin"

# url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}"

# headers = {
#     "X-Riot-Token": api_key
# }

# response = requests.get(url, headers=headers)

# if response.status_code == 200:
#     data = response.json()
#     puuid = data["puuid"]
#     print(f"Your puuid is: {puuid}")
# else:
#     print("An error occurred:", response.text)


In [ ]:
# region = "americas"

# # Get a list of match IDs for the player
# url = f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids"
# headers = {
#     "X-Riot-Token": api_key
# }
# response = requests.get(url, headers=headers)

# if response.status_code == 200:
#     match_ids = response.json()
# else:
#     print("An error occurred:", response.text)
#     exit()

# # Get the details for each match
# match_history = []
# for match_id in match_ids:
#     url = f"https://{region}.api.riotgames.com/tft/match/v1/matches/{match_id}"
#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         data = response.json()
#         match_history.append(data)
#     else:
#         print("An error occurred:", response.text)

# # Print the match history
# print(json.dumps(match_history, indent=2))


In [ ]:
# data = pd.json_normalize(match_history)

# df = pd.DataFrame(data)

# df.head()

## <center>Starting With The Data</center>

In this notebook, we will use Python 3 along with multiple packages such as pandas, numpy, scikit-learn, and more. Let's import everything we need, and read in the csv file we have.

In [21]:
import pandas as pd

csv = "tft_match_history.csv"

df = pd.read_csv(csv)

print(df)


C:\Users\SKaha\AppData\Local\Temp\ipykernel_12712\1856879481.py:5: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)


       placement        match_id                              augment0  \
0              5  NA1_4383295812             TFT6_Augment_SunfireBoard   
1              8  NA1_4383295812             TFT6_Augment_JeweledLotus   
2              7  NA1_4383295812             TFT7_Augment_MirageEmblem   
3              6  NA1_4383295812             TFT6_Augment_JeweledLotus   
4              3  NA1_4383295812  TFT7_Augment_SwiftshotPressTheAttack   
...          ...             ...                                   ...   
11651          6  NA1_4381248766       TFT7_Augment_DragonmancerEmblem   
11652          5  NA1_4381248766              TFT7_Augment_MikaelsGift   
11653          3  NA1_4381248766             TFT7_Augment_Preparation2   
11654          1  NA1_4381248766         TFT7_Augment_ScalescornEmblem   
11655          2  NA1_4381248766         TFT6_Augment_CyberneticShell2   

                               augment1                        augment2  \
0            TFT6_Augment_RadiantRel

Now we have a nice pandas dataframe to work with. Let's start cleaning up a little bit by getting rid of NaN values.

In [28]:
for name in df.select_dtypes("number"):
    df[name] = df[name].fillna(0)

for name in df.select_dtypes("object"):
    df[name] = df[name].fillna("None")

print(df)

       placement        match_id                              augment0  \
0              5  NA1_4383295812             TFT6_Augment_SunfireBoard   
1              8  NA1_4383295812             TFT6_Augment_JeweledLotus   
2              7  NA1_4383295812             TFT7_Augment_MirageEmblem   
3              6  NA1_4383295812             TFT6_Augment_JeweledLotus   
4              3  NA1_4383295812  TFT7_Augment_SwiftshotPressTheAttack   
...          ...             ...                                   ...   
11651          6  NA1_4381248766       TFT7_Augment_DragonmancerEmblem   
11652          5  NA1_4381248766              TFT7_Augment_MikaelsGift   
11653          3  NA1_4381248766             TFT7_Augment_Preparation2   
11654          1  NA1_4381248766         TFT7_Augment_ScalescornEmblem   
11655          2  NA1_4381248766         TFT6_Augment_CyberneticShell2   

                               augment1                        augment2  \
0            TFT6_Augment_RadiantRel